In [1]:
!python --version

Python 3.9.7


In [79]:
!pip install JPype1-1.3.0-cp39-cp39-win_amd64.whl
!pip install konlpy

Processing c:\users\user\movie_rating_analysis\jpype1-1.3.0-cp39-cp39-win_amd64.whl
JPype1 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [81]:
from konlpy.tag import Twitter
twitter = Twitter()

In [4]:
twitter.morphs('안녕 방금 데바데라는 게임을 하고 왔는데 내가 마지막 생존자였어 근데 결국 죽었어 아깝다')

['안녕',
 '방금',
 '데바',
 '데',
 '라는',
 '게임',
 '을',
 '하고',
 '왔는데',
 '내',
 '가',
 '마지막',
 '생존자',
 '였어',
 '근데',
 '결국',
 '죽었어',
 '아깝다']

## 댓글 수집하기

In [5]:
# 언차티드 분석
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=80737&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page={}'

import requests as req
from bs4 import BeautifulSoup

In [6]:
page = 1
res = req.get(url.format(1))
res

<Response [200]>

In [82]:
soup = BeautifulSoup(res.text, 'html.parser')

for li in soup.find('div', {'class':'score_result'}).find_all('li'):
    print('rating: ', li.em.text)
    print('comment: ', li.p.text.strip())

rating:  10
comment:  관람객


														
															
															
																톰홀랜드 앓이하고 있었는데 바로 또다른 영화로 볼 수 있어서 좋았어요 재밌게 잘 봤습니다
rating:  10
comment:  존좋 진짜 재미있다 진짜 인생영화
rating:  8
comment:  톰 홀랜드란 배우가  스파이더맨에 나왔었나  기억이 안나네언차티드에선  매력있네요
rating:  10
comment:  평점이 안 좋아서 기대 안 하고 봤는데 깊이 생각하지 않고 보기에 딱 좋아용 꿀잼
rating:  6
comment:  관람객


														
															
															
																내내 싱겁다가 그나마 마지막 액션신때문에 극장에서 볼 맛이 있었다
rating:  7
comment:  관람객


														
															
															
																그냥 적당히 킬링타임용으로 볼만함.
rating:  8
comment:  관람객


														
															
															
																가벼운 킬링 타임용 무비로 매우 괜춘~~
rating:  2
comment:  1. 영화에 재미는 처음 액션씬과 여주가 물에서 나오는 씬이 다 였다.2. 보물을 찾아가면서 숨겨진 암호를 푸는 과정이 너무 쉬웠다. 일반사람도 찾을 정도로...그동안 못찾은 이유를 모르겠다.3. 전투씬이 무슨 장난...
rating:  10
comment:  톰홀랜드 아니었으면 6점, 톰홀랜드라서 10점! 캬캬
rating:  6
comment:  관람객


														
															
															
																제작 전에 감독이 계속 바꼈던 영화 답게 충분히 더 잘 만들을 수 있을 것 같은데 

## 댓글 수집하기

In [17]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=80737&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page={}'

import requests as req
from bs4 import BeautifulSoup
import re

In [27]:
def get_comments(page = 1):
    res = req.get(url.format(page))
    soup = BeautifulSoup(res.text, 'html.parser')

    score, comment = [], []
    
    for li in soup.find('div', {'class':'score_result'}).find_all('li'):
        # 점수가 8점 이상이면 긍정(1), 5점 이하면 부정(0)
        if int(li.em.text) >= 8:
            score.append(1)
            c = re.sub('[\n\t\r관람객]', '', li.p.text.strip())
            comment.append(c)
        elif int(li.em.text) <= 5:
            score.append(0)
            c = re.sub('[\n\t\r관람객]', '', li.p.text.strip())
            comment.append(c)
            
    return score, comment

In [22]:
get_comments(1)

([1, 1, 1, 1, 1, 0, 1],
 ['톰홀랜드 앓이하고 있었는데 바로 또다른 영화로 볼 수 있어서 좋았어요 재밌게 잘 봤습니다',
  '존좋 진짜 재미있다 진짜 인생영화',
  '톰 홀랜드란 배우가  스파이더맨에 나왔었나  기억이 안나네언차티드에선  매력있네요',
  '평점이 안 좋아서 기대 안 하고 봤는데 깊이 생각하지 않고 보기에 딱 좋아용 꿀잼',
  '가벼운 킬링 타임용 무비로 매우 괜춘~~',
  '1. 영화에 재미는 처음 액션씬과 여주가 물에서 나오는 씬이 다 였다.2. 보물을 찾아가면서 숨겨진 암호를 푸는 과정이 너무 쉬웠다. 일반사도 찾을 정도로...그동안 못찾은 이유를 모르겠다.3. 전투씬이 무슨 장난...',
  '톰홀랜드 아니었으면 6점, 톰홀랜드라서 10점! 캬캬'])

In [28]:
import time

score, comment = [], []

for i in range(1,200):
    time.sleep(1)
    print(i, '번째 페이지의 댓글을 수집하고 있습니다.', end='\r')
    s, c = get_comments(i)
    score += s
    comment += c

In [36]:
import pandas as pd

df = pd.DataFrame([score, comment]).T
df.columns = ['score', 'comment']
df

,score,comment
0,1,톰홀랜드 앓이하고 있었는데 바로 또다른 영화로 볼 수 있어서 좋았어요 재밌게 잘 봤습니다
1,1,존좋 진짜 재미있다 진짜 인생영화
2,1,톰 홀랜드란 배우가 스파이더맨에 나왔었나 기억이 안나네언차티드에선 매력있네요
3,1,평점이 안 좋아서 기대 안 하고 봤는데 깊이 생각하지 않고 보기에 딱 좋아용 꿀잼
4,1,가벼운 킬링 타임용 무비로 매우 괜춘~~
...,...,...
1516,1,
1517,1,톰홀랜드가 보여줄 수 있는 딱 킬링타임용 영화ㅋㅋㅋ확실히 배우가 좋다 보니까 평타 ...
1518,1,게임원작은 안해봤지만 킬탐으론 딱 적당한듯. 스파때랑은 다른 톰 홀랜드의 청년미.....
1519,1,"원작 팬분들에겐 가벼운 분위기가 낯설 수 있는데, 톰 홀랜드 스타일을 좋아하는 분들..."


In [34]:
df.to_csv('test.csv')

## train/test set 분리하기

In [43]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(comment, score, test_size=0.2, random_state=0)

In [40]:
len(train_x), len(train_y)

(1216, 1216)

In [41]:
len(test_x), len(test_y)

(305, 305)

In [83]:
train_x[:9]

['남주는 잘생겼지만 어딘가 시시하고 허접한 인디아나존스',
 '볼거리 많고 톰홀랜드 귀여운 맛에 재밌게 본 킬림타임용 영화ㅎㅎ 예상이 가는 조금 뻔한 스토리였지만 볼만 했어요',
 '톰홀랜드인데 말해뭐해',
 '통신사 공짜 권으로 보고왔는데...돈주고 보기엔 글쎄...?',
 "재밌었음. 지루할 틈 없었고 '네셔널트레져'류의 보물찾기영화. 쿠키있음. 후속 나올듯?",
 '톰홀랜드 진짜 잘생기고 액션 넘 조아씀ㅋㅋㅋㅋ킬링타임으로 보기 딱 좋음',
 '',
 '재미있게 잘 봤습니다. *^^*',
 '톰홀랜드가 보여줄 수 있는 딱 킬링타임용 영화ㅋㅋㅋ확실히 배우가 좋다 보니까 평타 이상은 하는 듯 액션이 좋았음']

In [48]:
from konlpy.tag import Twitter
twitter = Twitter()

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(tokenizer=twitter.morphs, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x

<1216x1259 sparse matrix of type '<class 'numpy.float64'>'
	with 21155 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params={'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [52]:
grid_cv.best_params_

{'C': 7}

In [53]:
grid_cv.best_score_

0.8651315789473684

In [55]:
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x, test_y)

0.8491803278688524

In [56]:
a=['재밌어요']
my_comment = tfv.transform(a)
grid_cv.best_estimator_.predict(my_comment)

array([1])

In [77]:
b=['별로', '그저 그렇다','꼭 보세요','노잼', '미쳤다', '역시 톰홀랜드']
my_comment2 = tfv.transform(b)
grid_cv.best_estimator_.predict(my_comment2)

array([0, 0, 1, 0, 1, 1])